In [1]:
import gpxpy.gpx
import pandas as pd
import numpy as np
from haversine import haversine
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
data = []
gpx = gpxpy.parse(open('data/activity_10044912854.gpx'))
track = gpx.tracks[0]
segment = track.segments[0]

for point_idx, point in enumerate(segment.points):
    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])

columns = ['longitude', 'latitude', 'elevation', 'time', 'speed']
df = pd.DataFrame(data, columns=columns) 

In [3]:
point_dataframe = df

In [4]:
point_dataframe.describe()

,longitude,latitude,elevation,speed
count,4588.000000,4588.000000,4588.000000,4588.000000
mean,37.427840,55.815432,124.328029,0.706996
std,0.003573,0.001443,1.460919,0.686862
min,37.422542,55.812194,121.000000,0.000000
25%,37.424888,55.814820,123.199997,0.000000
50%,37.427363,55.815792,124.400002,0.314234
75%,37.430858,55.816638,125.599998,1.368805
max,37.434797,55.817410,128.000000,3.648887


In [5]:
point_dataframe.isna().sum()

longitude    0
latitude     0
elevation    0
time         0
speed        0
dtype: int64

In [6]:
point_dataframe['time'] = point_dataframe['time'].dt.tz_convert(tz='Europe/Moscow')

In [7]:

def line_plotter(df):
    subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
    fig = px.line(df, x='time', y="speed", render_mode="webgl",)
    fig2 = px.line(df, x='time', y="elevation", render_mode="webgl",)

    fig2.update_traces(yaxis="y2")

    subfig.add_traces(fig.data + fig2.data)
    subfig.layout.xaxis.title="Time"
    subfig.layout.yaxis.title="speed m/s"
    subfig.layout.yaxis2.title="elevation m"
# recoloring is necessary otherwise lines from fig und fig2 would share each color
    subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
    subfig.show()

line_plotter(point_dataframe)


In [8]:
(point_dataframe['time'] < pd.Timestamp('2022-11-27 13:37:13', tz='Europe/Moscow')).mean()

0.17000871839581516

In [9]:
def plot_df(df, zoom=14):
    fig = px.scatter_mapbox(
    df, 
    lat="latitude", 
    lon="longitude",
    hover_name="time",
    hover_data={
        "latitude": ":.2f",
        "longitude": ":.2f",
        "elevation": ":.2f"
    },
    zoom=zoom, 
    height=500,
)

    fig.update_layout(mapbox_style="open-street-map")


    fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>(%{customdata[0]:.2f}, %{customdata[1]:.2f})<br>Elev. %{customdata[2]:}.m<extra></extra>')

    fig.show()



In [10]:
def select_and_plot(df, parts, partnum):
    left_edge = parts[partnum]['from']
    right_edge = parts[partnum]['to']
    df_toplot= df[(df.time<=right_edge) & (df.time>=left_edge)]
    plot_df(df_toplot)
    line_plotter(df_toplot)

    


    


In [11]:
point_dataframe

,longitude,latitude,elevation,time,speed
0,37.434759,55.817366,123.599998,2022-11-27 13:24:13+03:00,1.314715
1,37.434766,55.817355,123.400002,2022-11-27 13:24:14+03:00,1.225948
2,37.434773,55.817346,123.199997,2022-11-27 13:24:15+03:00,1.194264
3,37.434775,55.817334,123.400002,2022-11-27 13:24:16+03:00,1.283032
4,37.434781,55.817323,123.199997,2022-11-27 13:24:17+03:00,1.488270
...,...,...,...,...,...
4583,37.433549,55.817076,122.199997,2022-11-27 14:40:36+03:00,1.860505
4584,37.433584,55.817071,122.400002,2022-11-27 14:40:37+03:00,1.867267
4585,37.433606,55.817066,122.599998,2022-11-27 14:40:38+03:00,1.346575
4586,37.433624,55.817063,122.599998,2022-11-27 14:40:39+03:00,1.383126


In [104]:
parts = {0:{"from": pd.Timestamp('2022-11-27 13:38:05', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 13:40:57', tz='Europe/Moscow')},
1:{"from": pd.Timestamp('2022-11-27 13:41:26', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 13:45:06', tz='Europe/Moscow')},
2:{"from": pd.Timestamp('2022-11-27 13:48:17', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 13:50:49', tz='Europe/Moscow')},
3:{"from": pd.Timestamp('2022-11-27 13:52:38', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 13:55:07', tz='Europe/Moscow')},
4:{"from": pd.Timestamp('2022-11-27 13:55:51', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 13:58:27', tz='Europe/Moscow')},
5:{"from": pd.Timestamp('2022-11-27 14:00:23', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 14:02:53', tz='Europe/Moscow')},
6:{"from": pd.Timestamp('2022-11-27 14:03:27', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 14:06:04', tz='Europe/Moscow')},
7:{"from": pd.Timestamp('2022-11-27 14:09:08', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 14:12:19', tz='Europe/Moscow')},
8:{"from": pd.Timestamp('2022-11-27 14:17:04', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 14:19:45', tz='Europe/Moscow')},
9:{"from": pd.Timestamp('2022-11-27 14:23:17', tz='Europe/Moscow'), "to":pd.Timestamp('2022-11-27 14:25:08', tz='Europe/Moscow')},


}
select_and_plot(point_dataframe, parts=parts, partnum=(5-1))

In [41]:
resultdict = {}

In [62]:
def select_and_mean(df, parts, partnum):
    left_edge = parts[partnum]['from']
    right_edge = parts[partnum]['to']
    df_toplot= df[(df.time<=right_edge) & (df.time>=left_edge)]
    
    return df_toplot[['longitude', 'latitude', 'elevation']].mean()

In [84]:
resultdict [1] =select_and_mean(point_dataframe, parts=parts, partnum=0)
resultdict [2] =select_and_mean(point_dataframe, parts=parts, partnum=1)
resultdict [3] =select_and_mean(point_dataframe, parts=parts, partnum=2)
resultdict [4] =select_and_mean(point_dataframe, parts=parts, partnum=3)
resultdict [5] =select_and_mean(point_dataframe, parts=parts, partnum=4)
resultdict [6] =select_and_mean(point_dataframe, parts=parts, partnum=5)
resultdict [7] =select_and_mean(point_dataframe, parts=parts, partnum=6)
resultdict [8] =select_and_mean(point_dataframe, parts=parts, partnum=7)
resultdict [9] =select_and_mean(point_dataframe, parts=parts, partnum=8)
resultdict [10] =select_and_mean(point_dataframe, parts=parts, partnum=9)

resultdict

{1: longitude     37.428495
 latitude      55.815792
 elevation    126.828902
 dtype: float64,
 2: longitude     37.428573
 latitude      55.816027
 elevation    126.150226
 dtype: float64,
 3: longitude     37.425849
 latitude      55.816876
 elevation    124.728105
 dtype: float64,
 4: longitude     37.425000
 latitude      55.816589
 elevation    125.089332
 dtype: float64,
 5: longitude     37.424794
 latitude      55.816741
 elevation    125.117198
 dtype: float64,
 6: longitude     37.424129
 latitude      55.815494
 elevation    125.607947
 dtype: float64,
 7: longitude     37.423786
 latitude      55.815307
 elevation    125.996203
 dtype: float64,
 8: longitude     37.422598
 latitude      55.813428
 elevation    124.357291
 dtype: float64,
 9: longitude     37.425657
 latitude      55.812197
 elevation    123.634567
 dtype: float64,
 10: longitude     37.427365
 latitude      55.814498
 elevation    122.276785
 dtype: float64}

In [97]:
correction = np.mean([56,56,56,55,53]) - np.mean([value[1].values[2] for value in resultdict.items()])

In [100]:
for key, value in resultdict.items():
    print(key)
    print(value.index.values)
    interm = value.values.copy()
    interm[2] += correction
    print(interm)

1
['longitude' 'latitude' 'elevation']
[37.42849548 55.81579187 57.05024628]
2
['longitude' 'latitude' 'elevation']
[37.42857296 55.81602692 56.3715703 ]
3
['longitude' 'latitude' 'elevation']
[37.4258492  55.81687559 54.94944944]
4
['longitude' 'latitude' 'elevation']
[37.42500017 55.81658921 55.31067642]
5
['longitude' 'latitude' 'elevation']
[37.42479418 55.81674088 55.33854211]
6
['longitude' 'latitude' 'elevation']
[37.4241287  55.81549413 55.82929154]
7
['longitude' 'latitude' 'elevation']
[37.42378622 55.81530708 56.21754709]
8
['longitude' 'latitude' 'elevation']
[37.42259777 55.81342753 54.57863575]
9
['longitude' 'latitude' 'elevation']
[37.42565675 55.81219699 53.85591185]
10
['longitude' 'latitude' 'elevation']
[37.42736457 55.81449818 52.49812921]


In [87]:
for key, value in resultdict.items():
    
    print(value.values[2])

126.82890183112525
126.15022585079141
124.72810498405906
125.08933197021484
125.11719765632775
125.60794709060366
125.99620263787764
124.35729130109151
123.63456740202727
122.27678476061139


55.2